<a href="https://colab.research.google.com/github/EAFIT-BI/Supervised-Learning-2025-I/blob/main/Modelos_de_ensamble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos de ensamble

Los modelos de ensamble utilizan la unión de varios modelos para mejorar el desempeño individual. Dentro los modelos generales de ensamble encontramos:

- Bagging (bootstrap)
- Bosques aleatorios (Random Forest)
- Boosting

In [ ]:
import pandas as pd
# Leemos los datos
datos = pd.read_csv('auto-mpg.data-original',
                    sep = '\s+', header = None)
datos = datos.dropna()
# Asignamos los nombres a las columnas
datos.columns = ['mpg', 'cylinders', 'displacement',
                'horsepower', 'weight', 'acceleration',
                'model year', 'origin', 'car name']
# Cambiamos el índice por defecto
datos.set_index('car name', inplace = True)
# Visualizamos la información modificada
datos.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
car name,,,,,,,,
chevrolet chevelle malibu,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
buick skylark 320,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
plymouth satellite,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
amc rebel sst,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
ford torino,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0


# Clasificador lineal

En este caso usaremos un regresor logístico para la clasificación.

In [ ]:
# Separamos las variables predictoras de la variable objetivo
X = datos.drop('origin', axis = 1)
y = datos['origin']

In [ ]:
# Particionamos en train y test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = 27,
                         stratify = y)

In [ ]:
# Preprocesamos los datos
from sklearn.preprocessing import StandardScaler
# Instanciamos el escalador
escalador = StandardScaler()
# Aplicamos al train y al test
escalador.fit_transform(X_train);

In [ ]:
escalador.fit_transform(X_test);

In [ ]:
from sklearn.linear_model import LogisticRegression
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
# El modelo de regresión logística tiene dos hiperparámetros
# a considerar. La C y la penalización.
modelo = LogisticRegression(max_iter = 500,
                            solver = 'liblinear',
                           class_weight = 'balanced')

# Instanciamos la sintonización
distribucion = {'C': loguniform(0.001, 10),
            'penalty': ['l1', 'l2']}
grid_search = RandomizedSearchCV(estimator = modelo,
                                param_distributions = distribucion,
                                scoring = 'f1_weighted',
                                cv = 5)

In [ ]:
# Hacemos la búsqueda
grid_search.fit(X_train, y_train)

# Visualizamos el mejor valor con los mejores hiperparámetros
print(f'Score y parámetros: {grid_search.best_score_, grid_search.best_params_}')

Score y parámetros: (np.float64(0.7749597625994435), {'C': np.float64(3.0279422506245397), 'penalty': 'l2'})


In [ ]:
# Imprimimos los reportes de clasificación tanto para el train
# como para el test

from sklearn.metrics import classification_report

y_pred = grid_search.predict(X_train)
print(classification_report(y_train, y_pred))

y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         1.0       0.96      0.85      0.90       184
         2.0       0.61      0.53      0.57        51
         3.0       0.53      0.78      0.63        59

    accuracy                           0.78       294
   macro avg       0.70      0.72      0.70       294
weighted avg       0.81      0.78      0.79       294

              precision    recall  f1-score   support

         1.0       0.93      0.92      0.93        61
         2.0       0.44      0.41      0.42        17
         3.0       0.55      0.60      0.57        20

    accuracy                           0.77        98
   macro avg       0.64      0.64      0.64        98
weighted avg       0.77      0.77      0.77        98



## Árboles

Dado que el clasificador lineal no tiene un desempeño óptimo, intentaremos hacer la clasificación con un modelo de árboles.

In [ ]:
# Tomamos los datos originales
X = datos.drop('origin', axis = 1)
y = datos['origin']

# Particionamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.75,
                                                    random_state = 27,
                                                    stratify = y)

In [ ]:
# Para este modelo, configuraremos la poda con ccp alpha.
# Para ello haremos un sintonización con el fin de determinar el
# valor de alpha.

from sklearn.tree import DecisionTreeClassifier
from scipy.stats import loguniform
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
# Instanciamos el modelo
modelo = DecisionTreeClassifier(random_state = 27)

# Definimos los parámetros de la sintonización
distribucion = {'ccp_alpha': loguniform(0.001, 10)}

# Instanciamos la sintonización
grid_search = RandomizedSearchCV(estimator = modelo,
                                param_distributions = distribucion,
                                scoring = 'f1_weighted',
                                cv = 5)
# Ejecutamos la sintonización
grid_search.fit(X_train, y_train)

# Visualizamos el mejor valor con los mejores hiperparámetros
print(f'Score y parámetros: {grid_search.best_score_,
                             grid_search.best_params_}')

Score y parámetros: (np.float64(0.7877727887070838), {'ccp_alpha': np.float64(0.004020266749573214)})


In [ ]:
# Imprimimos los reportes de clasificación con el fin
# de comparar los resultados con el modelo lineal

print(classification_report(y_train, grid_search.predict(X_train)))
print(classification_report(y_test, grid_search.predict(X_test)))

              precision    recall  f1-score   support

         1.0       0.98      0.97      0.98       184
         2.0       0.98      0.94      0.96        51
         3.0       0.92      0.98      0.95        59

    accuracy                           0.97       294
   macro avg       0.96      0.97      0.96       294
weighted avg       0.97      0.97      0.97       294

              precision    recall  f1-score   support

         1.0       0.92      0.95      0.94        61
         2.0       0.75      0.53      0.62        17
         3.0       0.78      0.90      0.84        20

    accuracy                           0.87        98
   macro avg       0.82      0.79      0.80        98
weighted avg       0.86      0.87      0.86        98



## Ensambles

Ahora compararemos los modelos aplicados con los ensambles, para comprobar si hay mejora o no en el sobre-entrenamiento.

### Bagging (bootstrap)

In [ ]:
# Implementamos el modelo de ensamble de Bagging
from sklearn.ensemble import BaggingClassifier

# Instanciamos el ensamble
ensamble = BaggingClassifier(estimator = DecisionTreeClassifier(),
                             n_estimators = 100,random_state = 27)

 #Entrenamos el modelo
ensamble.fit(X_train, y_train)

# Imprimimos los reportes de clasificación
print(classification_report(y_train, ensamble.predict(X_train)))
print(classification_report(y_test, ensamble.predict(X_test)))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       184
         2.0       1.00      1.00      1.00        51
         3.0       1.00      1.00      1.00        59

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294

              precision    recall  f1-score   support

         1.0       0.92      0.97      0.94        61
         2.0       0.92      0.65      0.76        17
         3.0       0.86      0.95      0.90        20

    accuracy                           0.91        98
   macro avg       0.90      0.85      0.87        98
weighted avg       0.91      0.91      0.90        98



### Bosques aleatorios (RandomForest - RF)

In [ ]:
# El segundo modelo de ensamble que probaremos serán los
# bosques aleatorios
from sklearn.ensemble import RandomForestClassifier

# Instanciamos el modelo
ensamble = RandomForestClassifier(n_estimators = 100,
                                  random_state = 27)
# Entrenamos el ensamble
ensamble.fit(X_train, y_train)

# Imprimimos los reportes de clasificación
print(classification_report(y_train, ensamble.predict(X_train)))
print(classification_report(y_test, ensamble.predict(X_test)))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       184
         2.0       1.00      1.00      1.00        51
         3.0       1.00      1.00      1.00        59

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294

              precision    recall  f1-score   support

         1.0       0.91      0.95      0.93        61
         2.0       0.67      0.59      0.62        17
         3.0       0.84      0.80      0.82        20

    accuracy                           0.86        98
   macro avg       0.81      0.78      0.79        98
weighted avg       0.85      0.86      0.85        98



### Boosting (Gradient Boosting)

In [ ]:
# El último modelo de ensamble que probaremos será el de
# boosting (gradient boosting)
from sklearn.ensemble import GradientBoostingClassifier

# Instanciamos el ensamble
ensamble = GradientBoostingClassifier(n_estimators = 100,
                                      random_state = 27)
# Entrenamos el ensamble
ensamble.fit(X_train, y_train)

# Imprimimos los reportes de clasificación
print(classification_report(y_train, ensamble.predict(X_train)))
print(classification_report(y_test, ensamble.predict(X_test)))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       184
         2.0       1.00      1.00      1.00        51
         3.0       1.00      1.00      1.00        59

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294

              precision    recall  f1-score   support

         1.0       0.94      0.97      0.95        61
         2.0       0.82      0.53      0.64        17
         3.0       0.79      0.95      0.86        20

    accuracy                           0.89        98
   macro avg       0.85      0.82      0.82        98
weighted avg       0.89      0.89      0.88        98

